In [152]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
import requests

In [153]:
pi = 0
pf = 13
main_url = "https://investigadores.unison.mx/es/persons/?format=&page="
numbers = list(range(pi,pf+1))
numbers

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [154]:
pages = []
for n in numbers:
    pages.append(main_url+str(n))
pages

['https://investigadores.unison.mx/es/persons/?format=&page=0',
 'https://investigadores.unison.mx/es/persons/?format=&page=1',
 'https://investigadores.unison.mx/es/persons/?format=&page=2',
 'https://investigadores.unison.mx/es/persons/?format=&page=3',
 'https://investigadores.unison.mx/es/persons/?format=&page=4',
 'https://investigadores.unison.mx/es/persons/?format=&page=5',
 'https://investigadores.unison.mx/es/persons/?format=&page=6',
 'https://investigadores.unison.mx/es/persons/?format=&page=7',
 'https://investigadores.unison.mx/es/persons/?format=&page=8',
 'https://investigadores.unison.mx/es/persons/?format=&page=9',
 'https://investigadores.unison.mx/es/persons/?format=&page=10',
 'https://investigadores.unison.mx/es/persons/?format=&page=11',
 'https://investigadores.unison.mx/es/persons/?format=&page=12',
 'https://investigadores.unison.mx/es/persons/?format=&page=13']

In [155]:
pages[0]

'https://investigadores.unison.mx/es/persons/?format=&page=0'

In [156]:
links = []
for page in pages:
    url = page
    url_get = requests.get(url)
    html = BeautifulSoup(url_get.content, "html.parser")
    for i in html.find_all("h3", class_="title"):
        links.append(i.a["href"])

In [ ]:
len(links)

50

In [ ]:
df = pd.DataFrame(columns=["nombre", "correo", "publicaciones", "citas", "indice h", "link"])
df

,nombre,correo,publicaciones,citas,indice h,link


In [ ]:
for link in links:
    url = link +"/publications/"
    url_get = requests.get(url)
    html = BeautifulSoup(url_get.content, 'html.parser')

    nombre = html.find("h1").text

    try:
        correo = html.find("li", class_="emails").a["href"]
        correo = correo.replace("mailto:","")
    except:
        correo = np.nan

    try:
        citas = html.find("li", class_="citations").span.span.text
    except:
        citas = np.nan
        
    try:
        h = html.find("li", class_="h-index").span.span.text
    except:
        h = np.nan
        
    try:
        publicaciones = html.find("a",class_="portal_link count increment-counter").span.text
        publicaciones = publicaciones.replace("\n","")
        publicaciones = publicaciones.replace(" ","")
    except:
        publicaciones = np.nan
        
    person = pd.DataFrame(columns=df.columns)
    person.loc[0] = [nombre, correo, publicaciones, citas, h, link]
    person
        
    df = df.append(person)

df.reset_index(drop=True, inplace=True)

In [ ]:
df

,nombre,correo,publicaciones,citas,indice h,link
0,Raúl Aceves Torres,raul.aceves@unison.mx,19,143,8,https://investigadores.unison.mx/es/persons/ra...
0,Milka del Carmen Acosta Enríquez,milka.acosta@unison.mx,25,44,4,https://investigadores.unison.mx/es/persons/mi...
0,Andrés Acosta Félix,andres.acosta@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/an...
0,Heriberto Acuña Campa,heriberto.acuna@unison.mx,6,105,5,https://investigadores.unison.mx/es/persons/he...
0,Karla Fabiola Acuña Meléndrez,karla.acuna@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/ka...
0,Manuel Adrian Acuña Zegarra,adrian.acuna@unison.mx,5,27,1,https://investigadores.unison.mx/es/persons/ma...
0,Norma Patricia Adan Bante,patricia.adan@unison.mx,8,86,3,https://investigadores.unison.mx/es/persons/no...
0,Milagros del Rosario Aguilar Martínez,NaN,1,0,0,https://investigadores.unison.mx/es/persons/mi...
0,Patricia Aguilar Talamante,patricia.aguilar@unison.mx,NaN,NaN,NaN,https://investigadores.unison.mx/es/persons/pa...
0,Dante Alberto Alcantara Bojorge,dante.alcantara@unison.mx,3,0,0,https://investigadores.unison.mx/es/persons/da...
